In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Dataset:  covtype.csv

Source: Remote Sensing and GIS Program, Department of Forest Sciences, College of Natural Resources, Colorado State University

Description: Predicting forest cover type from cartographic variables only (no remotely sensed data). The actual forest cover type for a given observation (30 x 30 meter cell) was determined from US Forest Service (USFS) Region 2 Resource Information System (RIS) data. Independent variables were derived from data originally obtained from US Geological Survey (USGS) and USFS data. Data is in raw form (not scaled) and contains binary (0 or 1) columns of data for qualitative independent variables (wilderness areas and soil types).

This study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. These areas represent forests with minimal human-caused disturbances, so that existing forest cover types are more a result of ecological processes rather than forest management practices.

Variables/Columns

- Elevation: Elevation in meters
- Aspect: Aspect in degrees azimuth
- Slope: Slope in degrees
- Horizontal_Distance_To_Hydrology: Horz Dist to nearest surface water features
- Vertical_Distance_To_Hydrology: Vert Dist to nearest surface water features
- Horizontal_Distance_To_Roadways: Horz Dist to nearest roadway
- Hillshade_9am: Hillshade index at 9am, summer solstice
- Hillshade_Noon: Hillshade index at noon, summer soltice
- Hillshade_3pm: Hillshade index at 3pm, summer solstice
- Horizontal_Distance_To_Fire_Points: Horz Dist to nearest wildfire ignition points
- Wilderness_Area: 0 (absence) or 1 (presence)
- Cover_Type: (2 types) Forest Cover Type designation
    - 0: Spruce/Fir
    - 1: Lodgepole Pine

In [2]:
import gdown
file_id = '12IKz_R30JLFmcqCl-iv_8Yjn5xDnwrKI'
output = 'covtype.csv'
gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=12IKz_R30JLFmcqCl-iv_8Yjn5xDnwrKI
To: /content/covtype.csv
100%|██████████| 2.15M/2.15M [00:00<00:00, 99.6MB/s]


'covtype.csv'

In [3]:
# Read the forest cover dataset
df = pd.read_csv('covtype.csv')
X = df.drop('cover', axis=1)
y = df['cover']
target_names = ["Spruce/Fir", "Lodgepole Pine"]

In [4]:
# Prepare the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# Import a Random Forests classifier
from sklearn.ensemble import RandomForestClassifier

In [6]:
# Fit a model, and then print a classification report
clf1 = RandomForestClassifier(random_state=1).fit(X_train_scaled, y_train)
y_pred = clf1.predict(X_test_scaled)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f'Training Score: {clf1.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf1.score(X_test_scaled, y_test)}')

                precision    recall  f1-score   support

    Spruce/Fir       0.88      0.86      0.87      5346
Lodgepole Pine       0.90      0.91      0.90      7033

      accuracy                           0.89     12379
     macro avg       0.89      0.89      0.89     12379
  weighted avg       0.89      0.89      0.89     12379

Training Score: 1.0
Testing Score: 0.8907827772841101


In [7]:
# Import an Extremely Random Trees classifier
from sklearn.ensemble import ExtraTreesClassifier

In [8]:
clf2 = ExtraTreesClassifier(random_state=1).fit(X_train_scaled, y_train)
y_pred = clf2.predict(X_test_scaled)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f'Training Score: {clf2.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf2.score(X_test_scaled, y_test)}')

                precision    recall  f1-score   support

    Spruce/Fir       0.90      0.87      0.88      5346
Lodgepole Pine       0.90      0.93      0.91      7033

      accuracy                           0.90     12379
     macro avg       0.90      0.90      0.90     12379
  weighted avg       0.90      0.90      0.90     12379

Training Score: 1.0
Testing Score: 0.8999111398335892


In [9]:
# Import an Adaptive Boosting classifier
from sklearn.ensemble import AdaBoostClassifier

In [10]:
clf3 = AdaBoostClassifier(random_state=1).fit(X_train_scaled, y_train)
y_pred = clf3.predict(X_test_scaled)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f'Training Score: {clf3.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf3.score(X_test_scaled, y_test)}')

                precision    recall  f1-score   support

    Spruce/Fir       0.72      0.72      0.72      5346
Lodgepole Pine       0.79      0.79      0.79      7033

      accuracy                           0.76     12379
     macro avg       0.76      0.75      0.75     12379
  weighted avg       0.76      0.76      0.76     12379

Training Score: 0.7584015510555795
Testing Score: 0.7595928588738994


In [11]:
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[
        ('RandomForest', clf1), ('ExtraTrees', clf2), ('AdaBoost', clf3)], voting='hard')
eclf1 = eclf1.fit(X_train_scaled, y_train)
y_pred = eclf1.predict(X_test_scaled)

In [12]:
print(classification_report(y_test, y_pred, target_names=target_names))
print(f'Training Score: {eclf1.score(X_train_scaled, y_train)}')
print(f'Testing Score: {eclf1.score(X_test_scaled, y_test)}')

                precision    recall  f1-score   support

    Spruce/Fir       0.88      0.85      0.87      5346
Lodgepole Pine       0.89      0.91      0.90      7033

      accuracy                           0.89     12379
     macro avg       0.89      0.88      0.89     12379
  weighted avg       0.89      0.89      0.89     12379

Training Score: 1.0
Testing Score: 0.8885208821391065


In [ ]:
# BONUS
def model_tester(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = model.fit(X_train_scaled, y_train)
    print(classification_report(y_test, y_pred, target_names=target_names))
    print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
    print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

model_tester(AdaBoostClassifier(random_state=1, n_estimators=100), X, y)
model_tester(AdaBoostClassifier(random_state=1, n_estimators=200), X, y)
model_tester(AdaBoostClassifier(random_state=1, n_estimators=200, learning_rate=0.1), X, y)
model_tester(AdaBoostClassifier(random_state=1, n_estimators=500, learning_rate=0.1), X, y)
model_tester(AdaBoostClassifier(random_state=1, n_estimators=1000, learning_rate=0.1), X, y)
model_tester(AdaBoostClassifier(random_state=1, n_estimators=2000, learning_rate=0.1), X, y)

                precision    recall  f1-score   support

    Spruce/Fir       0.74      0.72      0.73      5346
Lodgepole Pine       0.79      0.81      0.80      7033

      accuracy                           0.77     12379
     macro avg       0.77      0.77      0.77     12379
  weighted avg       0.77      0.77      0.77     12379

Training Score: 0.7736697544161999
Testing Score: 0.7711446805073108
                precision    recall  f1-score   support

    Spruce/Fir       0.74      0.72      0.73      5346
Lodgepole Pine       0.79      0.81      0.80      7033

      accuracy                           0.77     12379
     macro avg       0.77      0.77      0.77     12379
  weighted avg       0.77      0.77      0.77     12379

Training Score: 0.7770088323998277
Testing Score: 0.772356410049277
                precision    recall  f1-score   support

    Spruce/Fir       0.74      0.72      0.73      5346
Lodgepole Pine       0.79      0.81      0.80      7033

      accuracy 